In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import json
import ccxt

In [2]:
def convert_tz(utc):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    utc = utc.replace(tzinfo=from_zone).astimezone(to_zone)
    
    return utc

### Config 

In [3]:
with open('_keys/binance_keys.json') as keys_file:
    keys_dict = json.load(keys_file)
exchange = ccxt.binance(keys_dict)

In [4]:
symbol = 'ETH/USDT'
timeframe = '15m'
since = None
limit = 100

### Ticker

In [5]:
ticker = exchange.fetch_ticker(symbol)

In [6]:
ticker['last']

2487.77

### OHLCV

In [7]:
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)

In [8]:
df = pd.DataFrame(ohlcv)
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Time'] = pd.to_datetime(df['Time'], unit = 'ms')
df['Time'] = df['Time'].apply(lambda x: convert_tz(x))

In [9]:
df

,Time,Open,High,Low,Close,Volume
0,2021-04-25 16:45:00+07:00,2205.17,2257.93,2203.36,2255.04,16658.32162
1,2021-04-25 17:00:00+07:00,2255.04,2288.72,2235.38,2282.51,21214.32605
2,2021-04-25 17:15:00+07:00,2282.12,2289.58,2267.99,2278.93,12978.45257
3,2021-04-25 17:30:00+07:00,2278.96,2281.35,2256.74,2265.27,11914.64678
4,2021-04-25 17:45:00+07:00,2265.19,2269.45,2250.03,2259.50,9095.96544
...,...,...,...,...,...,...
95,2021-04-26 16:30:00+07:00,2425.78,2448.79,2424.82,2448.18,5309.74615
96,2021-04-26 16:45:00+07:00,2448.18,2485.00,2446.69,2483.23,22540.86718
97,2021-04-26 17:00:00+07:00,2483.38,2485.00,2468.29,2479.04,13736.58850
98,2021-04-26 17:15:00+07:00,2479.04,2496.90,2473.29,2495.31,15250.75645


### Order

In [10]:
open_orders = exchange.fetch_open_orders(symbol)

In [11]:
open_orders

[{'info': {'symbol': 'ETHUSDT',
   'orderId': '3759243158',
   'orderListId': '-1',
   'clientOrderId': 'and_23bf96d909cc4f07b1bf37f08c5ff9ce',
   'price': '2600.00000000',
   'origQty': '0.32281000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'SELL',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619410920145',
   'updateTime': '1619410920145',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '3759243158',
  'clientOrderId': 'and_23bf96d909cc4f07b1bf37f08c5ff9ce',
  'timestamp': 1619410920145,
  'datetime': '2021-04-26T04:22:00.145Z',
  'lastTradeTimestamp': None,
  'symbol': 'ETH/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'sell',
  'price': 2600.0,
  'stopPrice': 0.0,
  'amount': 0.32281,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 0.32281,
  'status': 'open',
  'fee': None,


In [12]:
order_type = 'limit'
side = 'buy'
amount = 0.1
price = 1500

order = exchange.create_order(symbol, order_type, side, amount, price)

In [13]:
order

{'info': {'symbol': 'ETHUSDT',
  'orderId': '3762873269',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S8287f94c7d1ccaf84b59ed05',
  'transactTime': '1619433128006',
  'price': '1500.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'fills': []},
 'id': '3762873269',
 'clientOrderId': 'x-R4BD3S8287f94c7d1ccaf84b59ed05',
 'timestamp': 1619433128006,
 'datetime': '2021-04-26T10:32:08.006Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 1500.0,
 'stopPrice': None,
 'amount': 0.1,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 0.1,
 'status': 'open',
 'fee': None,
 'trades': [],
 'fees': []}

In [14]:
open_orders = exchange.fetch_open_orders(symbol)

In [15]:
open_orders

[{'info': {'symbol': 'ETHUSDT',
   'orderId': '3759243158',
   'orderListId': '-1',
   'clientOrderId': 'and_23bf96d909cc4f07b1bf37f08c5ff9ce',
   'price': '2600.00000000',
   'origQty': '0.32281000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'SELL',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619410920145',
   'updateTime': '1619410920145',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '3759243158',
  'clientOrderId': 'and_23bf96d909cc4f07b1bf37f08c5ff9ce',
  'timestamp': 1619410920145,
  'datetime': '2021-04-26T04:22:00.145Z',
  'lastTradeTimestamp': None,
  'symbol': 'ETH/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'sell',
  'price': 2600.0,
  'stopPrice': 0.0,
  'amount': 0.32281,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 0.32281,
  'status': 'open',
  'fee': None,


In [16]:
order = exchange.cancel_order (order['id'], symbol)

In [17]:
order

{'info': {'symbol': 'ETHUSDT',
  'origClientOrderId': 'x-R4BD3S8287f94c7d1ccaf84b59ed05',
  'orderId': '3762873269',
  'orderListId': '-1',
  'clientOrderId': 'b461RnwG0DHNoG9vU4plSz',
  'price': '1500.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'CANCELED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY'},
 'id': '3762873269',
 'clientOrderId': 'b461RnwG0DHNoG9vU4plSz',
 'timestamp': None,
 'datetime': None,
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 1500.0,
 'stopPrice': None,
 'amount': 0.1,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 0.1,
 'status': 'canceled',
 'fee': None,
 'trades': [],
 'fees': []}

In [18]:
open_orders = exchange.fetch_open_orders(symbol)

In [19]:
open_orders

[{'info': {'symbol': 'ETHUSDT',
   'orderId': '3759243158',
   'orderListId': '-1',
   'clientOrderId': 'and_23bf96d909cc4f07b1bf37f08c5ff9ce',
   'price': '2600.00000000',
   'origQty': '0.32281000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'SELL',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619410920145',
   'updateTime': '1619410920145',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '3759243158',
  'clientOrderId': 'and_23bf96d909cc4f07b1bf37f08c5ff9ce',
  'timestamp': 1619410920145,
  'datetime': '2021-04-26T04:22:00.145Z',
  'lastTradeTimestamp': None,
  'symbol': 'ETH/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'sell',
  'price': 2600.0,
  'stopPrice': 0.0,
  'amount': 0.32281,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 0.32281,
  'status': 'open',
  'fee': None,


In [20]:
test_symbol = 'LUNA/USDT'
closed_order = exchange.fetch_closed_orders(test_symbol)

In [21]:
closed_order

[{'info': {'symbol': 'LUNAUSDT',
   'orderId': '141612146',
   'orderListId': '-1',
   'clientOrderId': 'and_3b0684b6248f41349b5c24de5b06cbdf',
   'price': '13.63810000',
   'origQty': '49.16000000',
   'executedQty': '49.16000000',
   'cummulativeQuoteQty': '670.44899600',
   'status': 'FILLED',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'BUY',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619069139146',
   'updateTime': '1619069195122',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '141612146',
  'clientOrderId': 'and_3b0684b6248f41349b5c24de5b06cbdf',
  'timestamp': 1619069139146,
  'datetime': '2021-04-22T05:25:39.146Z',
  'lastTradeTimestamp': None,
  'symbol': 'LUNA/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'buy',
  'price': 13.6381,
  'stopPrice': 0.0,
  'amount': 49.16,
  'cost': 670.448996,
  'average': 13.6381,
  'filled': 49.16,
  'remaining': 0.0,
  'status': 'closed',
  '

In [22]:
test_id = '148291513'
status = [x for x in closed_order if x['id'] == test_id][0]['status']

### Balance

In [23]:
balance = exchange.fetch_balance()

In [24]:
pd.DataFrame(balance['info']['balances'])

,asset,free,locked
0,BTC,0.00000000,0.00000000
1,LTC,0.00000000,0.00000000
2,ETH,0.00000221,0.32281000
3,NEO,0.00000000,0.00000000
4,BNB,0.00000000,0.00000000
...,...,...,...
407,1INCHUP,0.00000000,0.00000000
408,1INCHDOWN,0.00000000,0.00000000
409,MIR,0.00000000,0.00000000
410,BAR,0.00000000,0.00000000


In [25]:
coin = symbol.split('/')[0]
balance[coin]

{'free': 2.21e-06, 'used': 0.32281, 'total': 0.32281221}